In [ ]:
#역시 CelebA 데이터셋을 이용하여


In [ ]:
#!mkdir data_faces && wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip

In [ ]:
!gdown --id 1Ewn2uWDzY9k9_00gCsoTsJk9AJveRLoJ

In [ ]:
import zipfile


with zipfile.Zipfile('list_attr_celeba.csv.zip', 'r') as zip_ref:
  zip_ref.extractall('./')
# 압축 풀어서 하위 폴더에 넣기!

In [ ]:
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torch.optim as optim
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset

from torch.autograd import Variable

import pandas as pd

In [ ]:
def show(img, renorm=False, nrow=8, interpolation='bicubic'):
  if renorm:
    img = img*0.5 + 0.5
  img_grid = torchvision.utils.make_grid(img, nrow=nrow).numpy()
  plt.figure()
  plt.imshow(np.transpose(img_grid, (1,2,0)), interpolation=interpolation) # transpose는 그림의 데이터 차원 순서를 바꾸기, RGB의 순서가 torch와 plt의 처리 순서가 다르기 때문...
  plt.axis('off')
  plt.show()

In [ ]:
#각 그림에 담긴 attribute-특성을 붙여야 함
df = pd.read_csv('list_attr_celeba.csv')
df.head()

In [ ]:
transform = transforms.Compose([
    transforms.Resize(size=(128, 128), interpolation=Image.BICUBIC),
    transforms.ToTensor()
])

In [ ]:
data, labels = df.values[:, 0], (df.values[:, 1:]+1)//2 #label은 -1과 1로 되어있어 해당 내용을 0, 1로 바꾸기

classes = df.columns[1:]
classes[[39,20,22,2,31,15]] # 그림으로 변경할 특성들 여섯개만 뽑기

In [ ]:
class CelebADataset(Dataset):
  def __init__(self, data, labels, classes, transform=None):
    self.data = data
    self.labels = labels
    self.classes = classes
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    img = Image.open(os.path.join('data_faces', 'img_align_celeba', self.data[idx])).convert('RGB')
    label = torch.Tensor(self.labels[idx,[39,20,22,2,31,15]].astype('uint8'))

    if self.transform:
      img = self.transform(img)
    sample = {'images': img, 'labels': label}
    return sample



In [ ]:
celeba_data = CelebADataset(data, labels, classes, transform)

In [ ]:
celeba_data[0]

In [ ]:
celeba_loader = DataLoader(celeba_data, batch_size=64, shuffle=True)

In [ ]:
batch= next(iter(celeba_loader))

show(batch['images'][0:16], renorm=True, nrow=4)

In [ ]:
class VAE(nn.Module):
  def __init__(self, image_size=128, latent_dim=512):
    super(VAE, self).__init__()

    self.encoder = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1),
        nn.ReLU(),
        nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
        nn.ReLU(),
        nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
        nn.ReLU(),
        nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
        nn.ReLU(),
        nn.Flatten()
    )
    # embedding 차원 정의 -
    self.Embeddings = nn.Embedding(2, 10) # 특성 표시 방법이 2가지 - 0, 1, 그리고 그림, 그리고 특성을 10개의 벡터 차원으로 표현
    # 평균, 분산과 관련된 파라미터 정의
    self.fc_mu = nn.Linear(256 * (image_size // 16) * (image_size // 16), latent_dim)
    self.fc_var = nn.Linear(256 * (image_size // 16) * (image_size // 16), latent_dim)

    self.decoder_input = nn.Linear(latent_dim+6*10, 256 * (image_size // 16) * (image_size // 16))
    # 위에서 decoder의 입력으로 들어가는 차원에 추가되는 이유는 우리가 변경할 특성의 숫자가 6개고, 또한 그 특성은 각각 embeddings에서 10의 차원으로 정의하였기 때문에, 각 특성의 갯수 * embedding의 차원을 고려하여 추가되는 것
    self.decoder = nn.Sequential(
        nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
        nn.ReLU(),
        nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
        nn.ReLU(),
        nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
        nn.ReLU(),
        nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1),
        nn.Sigmoid()
    )
    self.image_size = image_size

  def encode(self, x):
    x = self.encoder(x)
    mu, logvar = self.fc_mu(x), self.fc_var(x)
    return mu, logvar

  def reparameterize(self, mu, logvar):
    std = torch.exp(0.5*logvar)
    eps = torch.randn_like(std)
    return mu + eps*std

  def decode(self, z):
    x = self.decoder_input(z)
    x = x.view(-1, 256, (self.image_size // 16), (self.image_size // 16))
    x = self.decoder(x)
    return x

  def forward(self, x, labels):
    mu, logvar = self.encode(x)
    z = self.reparameterize(mu, logvar)
    B, _ = z.shape
    z_labels = self.Embeddings(labels).reshape(B, -1)
    z = torch.cat((z, z_labels), 1)
    recon_x = self.decode(z)
    return recon_x, mu, logvar



In [ ]:
def vae_loss(recon_x, x, mu, logvar):
  BCE = nn.BCELoss(reduction='sum')(recon_x, x)
  KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
  return BCE + KLD


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = VAE().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
num_epochs = 50

best_loss = np.inf
model.train()

for epoch in range(num_epochs):
  total_loss = 0
  cpt = 0
  for batch_idx, data in enumerate(celeba_loader):
    imgs, labels = data['images'].float().to(device), data['labels'].long().to(device)

    optimizer.zero_grad()
    recon_imgs, mu, logvar = model(imgs, labels)
    loss = vae_loss(recon_imgs, imgs, mu, logvar)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
    cpt += 1

    if batch_idx % 100 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(celeba_loader)}], Loss: {total_loss/cpt:4f}')
  print(f'Epoch [{epoch+1}/{num_epochs}], Total Loss: {total_loss/len(celeba_loader):.4f}')
  if total_loss < best_loss:
    best_loss = total_loss
    torch.save(model.state_dict(), 'celeba_vae.pth')

In [ ]:
# 위에서의 모델의 훈련이 끝나면...?

